In [16]:
import pandas as pd
import os
from datetime import datetime

os.makedirs("data/clean", exist_ok=True)

# === Kunddata ===
customers_df = pd.read_csv("data/sebank_customers_with_accounts.csv")
print(f"Totalt kunder innan validering: {len(customers_df)}")

# Ta bort rader utan Customer eller BankAccount
customers_df = customers_df.dropna(subset=["Customer", "BankAccount"])
print(f"Efter dropna på Customer och BankAccount: {len(customers_df)}")

# Telefon: tillåt siffror, mellanslag, +, bindestreck, 7-15 tecken
phone_pattern = r"^\+?[\d\s\-]{7,15}$"
customers_df = customers_df[customers_df["Phone"].str.contains(phone_pattern, na=False)]
print(f"Efter telefonfilter: {len(customers_df)}")

# Personnummer: 6 siffror + valfritt - eller + + 4 siffror
personnummer_pattern = r"^\d{6}[-+]?\d{4}$"
customers_df = customers_df[customers_df["Personnummer"].str.contains(personnummer_pattern, na=False)]
print(f"Efter personnummerfilter: {len(customers_df)}")

# Ta bort dubletter på BankAccount
customers_df = customers_df.drop_duplicates(subset="BankAccount")
print(f"Efter drop_duplicates på BankAccount: {len(customers_df)}")

# Spara ren kunddata
customers_df.to_csv("data/clean/customers_clean.csv", index=False)
print("Kunddata sparad i data/clean/customers_clean.csv\n")


# === Transaktionsdata ===
transactions_df = pd.read_csv("data/transactions.csv")
transactions_df["notes"] = transactions_df["notes"].fillna("ingen kommentar")
transactions_df["timestamp"] = pd.to_datetime(transactions_df["timestamp"], errors='coerce')

print(f"Totalt transaktioner innan validering: {len(transactions_df)}")

# Belopp >= 0.01
transactions_df = transactions_df[transactions_df["amount"] >= 0.01]
print(f"Efter filter på amount >= 0.01: {len(transactions_df)}")

# Valuta: exakt 3 versaler
transactions_df = transactions_df[transactions_df["currency"].str.match(r"^[A-Z]{3}$", na=False)]
print(f"Efter valutafilter: {len(transactions_df)}")

# Ta bort rader utan notes
transactions_df = transactions_df.dropna(subset=["notes"])
print(f"Efter dropna på notes: {len(transactions_df)}")

# Ta bort dubletter på transaction_id
transactions_df = transactions_df.drop_duplicates(subset="transaction_id")
print(f"Efter drop_duplicates på transaction_id: {len(transactions_df)}")

# Spara ren transaktionsdata
transactions_df.to_csv("data/clean/transactions_clean.csv", index=False)
print("Transaktionsdata sparad i data/clean/transactions_clean.csv\n")


# === Flagga misstänkta transaktioner ===
flagged = []

# 1. Samma konto som avsändare och mottagare
samma_konto = transactions_df[transactions_df["sender_account"] == transactions_df["receiver_account"]]
for _, row in samma_konto.iterrows():
    flagged.append({
        "transaction_id": row["transaction_id"],
        "reason": "Samma konto som avsändare och mottagare",
        "flagged_date": datetime.today().strftime("%Y-%m-%d"),
        "amount": row["amount"]
    })

# 2. Högt belopp
for _, row in transactions_df[transactions_df["amount"] > 100_000].iterrows():
    flagged.append({
        "transaction_id": row["transaction_id"],
        "reason": "Belopp över 100 000 SEK",
        "flagged_date": datetime.today().strftime("%Y-%m-%d"),
        "amount": row["amount"]
    })

# 3. Negativt belopp (ska inte finnas kvar, men dubbelkolla)
for _, row in transactions_df[transactions_df["amount"] < 0].iterrows():
    flagged.append({
        "transaction_id": row["transaction_id"],
        "reason": "Negativt belopp",
        "flagged_date": datetime.today().strftime("%Y-%m-%d"),
        "amount": row["amount"]
    })

# 4. Flera transaktioner från samma konto inom 1 minut
tx = transactions_df.sort_values(by=["sender_account", "timestamp"])
for account, group in tx.groupby("sender_account"):
    timestamps = group["timestamp"].tolist()
    for i in range(len(timestamps) - 3):
        if (timestamps[i + 3] - timestamps[i]).total_seconds() <= 60:
            match = group.iloc[i:i+4]
            for _, row in match.iterrows():
                flagged.append({
                    "transaction_id": row["transaction_id"],
                    "reason": "Flera transaktioner inom 1 minut",
                    "flagged_date": datetime.today().strftime("%Y-%m-%d"),
                    "amount": row["amount"]
                })

# Ta bort eventuella dubbletter i flaggade transaktioner
flagged_df = pd.DataFrame(flagged).drop_duplicates(subset=["transaction_id", "reason"])

# Spara flaggade transaktioner
flagged_df.to_csv("data/clean/flagged_transactions.csv", index=False)
print("Flaggade transaktioner sparade i data/clean/flagged_transactions.csv\n")

print("Klart! Validering och flaggning klar.")








Error while processing DataContextConfig: datasources notebooks validations_store_name evaluation_parameter_store_name include_rendered_content anonymous_usage_statistics
Encountered errors during loading config.  See ValidationError for more details.


InvalidDataContextConfigError: Error while processing DataContextConfig: datasources notebooks validations_store_name evaluation_parameter_store_name include_rendered_content anonymous_usage_statistics

AttributeError: 'FileDataContext' object has no attribute 'sources'

ImportError: cannot import name 'BaseDataContext' from 'great_expectations.data_context' (c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\__init__.py)

SyntaxError: invalid syntax (246434187.py, line 2)

TypeError: DataContextConfig.__init__() got an unexpected keyword argument 'anonymous_usage_statistics'

✅ تم إنشاء مشروع Great Expectations بنجاح!


Error while processing DataContextConfig: datasources notebooks validations_store_name evaluation_parameter_store_name include_rendered_content anonymous_usage_statistics
Encountered errors during loading config.  See ValidationError for more details.


InvalidDataContextConfigError: Error while processing DataContextConfig: datasources notebooks validations_store_name evaluation_parameter_store_name include_rendered_content anonymous_usage_statistics

✅ تم إنشاء السياق بنجاح!


c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\abstract_data_context.py:821: DeprecationWarning: add_or_update_datasource() from the DataContext is deprecated and will be removed in a future version of GX. Please use `context.data_sources.add_or_update` instead.
  warnings.warn(


ValueError: Either datasource or kwargs are required

AttributeError: 'FileDataContext' object has no attribute 'sources'

c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\abstract_data_context.py:821: DeprecationWarning: add_or_update_datasource() from the DataContext is deprecated and will be removed in a future version of GX. Please use `context.data_sources.add_or_update` instead.
  warnings.warn(


AttributeError: 'dict' object has no attribute 'datasource_name'

c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\abstract_data_context.py:821: DeprecationWarning: add_or_update_datasource() from the DataContext is deprecated and will be removed in a future version of GX. Please use `context.data_sources.add_or_update` instead.
  warnings.warn(


AttributeError: 'DataFrameAsset' object has no attribute 'build_batch'

1.5.3


c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\serializable_data_context.py:159: UserWarning: Warning. An existing `great_expectations.yml` was found here: c:\Users\46722\Desktop\python-bank-projekt\gx_project.
    - No action was taken.
  warnings.warn(message)
c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\serializable_data_context.py:167: UserWarning: Warning. An existing `config_variables.yml` was found here:
            c:\Users\46722\Desktop\python-bank-projekt\gx_project\uncommitted. - No action was taken.
  warnings.warn(message)


TypeError: DataFrameAsset.get_batch() got an unexpected keyword argument 'dataframe'

ValidationError: 2 validation errors for BatchRequest
batch_identifiers
  extra fields not permitted (type=value_error.extra)
runtime_parameters
  extra fields not permitted (type=value_error.extra)

AttributeError: 'DataFrameAsset' object has no attribute 'get_batch_from_dataframe'

ModuleNotFoundError: No module named 'great_expectations.experimental.datasources'

✅ تم إنشاء السياق بنجاح!


DataContextError: Can not write the fluent datasource my_pandas_ds because a datasource of that name already exists in the data context.

TypeError: 'DataSourceManager' object is not subscriptable

c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\datasource\fluent\sources.py:644: DeprecationWarning: `delete_pandas` is deprecated as of v0.17.2 and will be removed in v0.19. Please use `.sources.delete` moving forward.
  warnings.warn(


KeyError: 'my_pandas_ds'

c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\serializable_data_context.py:159: UserWarning: Warning. An existing `great_expectations.yml` was found here: c:\Users\46722\Desktop\python-bank-projekt\gx_project.
    - No action was taken.
  warnings.warn(message)
c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\serializable_data_context.py:167: UserWarning: Warning. An existing `config_variables.yml` was found here:
            c:\Users\46722\Desktop\python-bank-projekt\gx_project\uncommitted. - No action was taken.
  warnings.warn(message)


TypeError: 'DataSourceManager' object is not subscriptable

c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\serializable_data_context.py:159: UserWarning: Warning. An existing `great_expectations.yml` was found here: c:\Users\46722\Desktop\python-bank-projekt\gx_project.
    - No action was taken.
  warnings.warn(message)
c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\serializable_data_context.py:167: UserWarning: Warning. An existing `config_variables.yml` was found here:
            c:\Users\46722\Desktop\python-bank-projekt\gx_project\uncommitted. - No action was taken.
  warnings.warn(message)
c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\abstract_data_context.py:914: DeprecationWarning: context.get_datasource is deprecated as of v1.2.0. Please use context.data_sources.get instead
  warnings.warn(


ValueError: "transactions_asset" already exists (all existing assets are transactions_asset)

TypeError: DataFrameAsset.build_batch_request() got an unexpected keyword argument 'dataframe'

c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\serializable_data_context.py:159: UserWarning: Warning. An existing `great_expectations.yml` was found here: c:\Users\46722\Desktop\python-bank-projekt\gx_project.
    - No action was taken.
  warnings.warn(message)
c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\serializable_data_context.py:167: UserWarning: Warning. An existing `config_variables.yml` was found here:
            c:\Users\46722\Desktop\python-bank-projekt\gx_project\uncommitted. - No action was taken.
  warnings.warn(message)
c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\abstract_data_context.py:914: DeprecationWarning: context.get_datasource is deprecated as of v1.2.0. Please use context.data_sources.get instead
  warnings.warn(


BuildBatchRequestError: Bad input to build_batch_request: options must contain exactly 1 key, 'dataframe'.

c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\serializable_data_context.py:159: UserWarning: Warning. An existing `great_expectations.yml` was found here: c:\Users\46722\Desktop\python-bank-projekt\gx_project.
    - No action was taken.
  warnings.warn(message)
c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\serializable_data_context.py:167: UserWarning: Warning. An existing `config_variables.yml` was found here:
            c:\Users\46722\Desktop\python-bank-projekt\gx_project\uncommitted. - No action was taken.
  warnings.warn(message)
c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\abstract_data_context.py:914: DeprecationWarning: context.get_datasource is deprecated as of v1.2.0. Please use context.data_sources.get instead
  warnings.warn(


AttributeError: 'FileDataContext' object has no attribute 'add_or_update_expectation_suite'

c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\serializable_data_context.py:159: UserWarning: Warning. An existing `great_expectations.yml` was found here: c:\Users\46722\Desktop\python-bank-projekt\gx_project.
    - No action was taken.
  warnings.warn(message)
c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\serializable_data_context.py:167: UserWarning: Warning. An existing `config_variables.yml` was found here:
            c:\Users\46722\Desktop\python-bank-projekt\gx_project\uncommitted. - No action was taken.
  warnings.warn(message)
c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\abstract_data_context.py:914: DeprecationWarning: context.get_datasource is deprecated as of v1.2.0. Please use context.data_sources.get instead
  warnings.warn(
c:\Users\46722\Desktop\python-bank-projekt\ven

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

✅ تم حفظ التوقعات بنجاح! 🎉


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

🔍 تم التحقق من البيانات بنجاح!
عدد التوقعات التي تم تنفيذها: 1


📄 افتح التقرير من هنا:
file://c:\Users\46722\Desktop\python-bank-projekt\gx_project\uncommitted/data_docs/local_site/index.html


In [1]:
import great_expectations as gx



c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\serializable_data_context.py:159: UserWarning: Warning. An existing `great_expectations.yml` was found here: c:\Users\46722\Desktop\python-bank-projekt\gx_project.
    - No action was taken.
  warnings.warn(message)
c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\serializable_data_context.py:167: UserWarning: Warning. An existing `config_variables.yml` was found here:
            c:\Users\46722\Desktop\python-bank-projekt\gx_project\uncommitted. - No action was taken.
  warnings.warn(message)
c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\abstract_data_context.py:914: DeprecationWarning: context.get_datasource is deprecated as of v1.2.0. Please use context.data_sources.get instead
  warnings.warn(


AttributeError: 'FileDataContext' object has no attribute 'add_or_update_checkpoint'

ImportError: cannot import name 'SimpleCheckpoint' from 'great_expectations.checkpoint' (c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\checkpoint\__init__.py)

c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\serializable_data_context.py:159: UserWarning: Warning. An existing `great_expectations.yml` was found here: c:\Users\46722\Desktop\python-bank-projekt\gx_project.
    - No action was taken.
  warnings.warn(message)
c:\Users\46722\Desktop\python-bank-projekt\venv311\Lib\site-packages\great_expectations\data_context\data_context\serializable_data_context.py:167: UserWarning: Warning. An existing `config_variables.yml` was found here:
            c:\Users\46722\Desktop\python-bank-projekt\gx_project\uncommitted. - No action was taken.
  warnings.warn(message)
